## Setup

In [ ]:
import sys
from pathlib import Path

from aocd import get_data, submit

In [4]:
# Add parent directory to path to allow relative imports into Jupyter notebook
sys.path.append(str(Path.cwd().parent))

In [5]:
# Get raw advent-of-code data
data: str = get_data(year=2024, day=16)

## Part a

In [ ]:
# Imports
from collections import defaultdict
from heapq import heappop, heappush

from common.utils.dict_grid import find_object_in_grid, print_grid, text_to_dict

In [ ]:
def find_shortest_path(
    grid: dict[complex, str],
    initial_direction: complex,
    *,
    find_all: bool = False,  # Added for part b: Flag to find all shortest paths instead of just one
) -> tuple[int | float, list[list[complex]]]:
    """Find shortest path(s) using A* search."""
    start = find_object_in_grid(grid, "S")
    end = find_object_in_grid(grid, "E")

    best_paths = []  # Added for part b: List to store all shortest paths
    best_score = float("inf")
    best_score_per_position = defaultdict(lambda: float("inf"))

    queue = [(0, t := 0, start, initial_direction, [start])]  # Score, tie-breaker, position, direction, path

    while queue:
        score, _, position, direction, path = heappop(queue)

        # Skip if this score is worse than previously found
        if score > best_score_per_position[(position, direction)]:
            continue
        # Otherwise, track score for this position
        best_score_per_position[(position, direction)] = score

        # Found end, track path if it's the best
        if position == end:
            # Added for part b: Logic to track all shortest paths
            if score < best_score:
                best_score = score
                best_paths = [path]
            elif score == best_score:
                best_paths.append(path)
            if not find_all:
                return best_score, [path]  # Early return for first shortest path
            continue

        for turn in [1j, 1, -1j]:
            # Find new position and direction
            new_direction = direction * turn
            new_position = position + new_direction

            # Skip if new position is invalid
            if grid.get(new_position, "#") == "#":
                continue

            # Calculate new score and update tie-breaker and queue
            new_score = score + 1 + (1000 if turn != 1 else 0)
            t += 1
            heappush(queue, (new_score, t, new_position, new_direction, [*path, new_position]))

    # Return all paths with minimum score
    return best_score, best_paths

In [ ]:
# Parse grid
grid = text_to_dict(data)

In [9]:
# Find shortest path
minimum_path_score, best_paths = find_shortest_path(grid, 1)

In [ ]:
# Print the grid with the shortest paths
grid_with_shortest_path = {k: "O" if k in best_paths[0] else v for k, v in grid.items()}
print(f"The best path has score of {minimum_path_score}.")
print_grid(grid_with_shortest_path)

In [ ]:
# Submit answer
submit(minimum_path_score, part="a", day=16, year=2024)

## Part b

In [11]:
# See added code in part a, denoted by '# Added for part b: ...'

# Find all shortest paths
minimum_path_score, best_paths = find_shortest_path(grid, 1, find_all=True)

In [ ]:
# Print the grid with the shortest paths
best_path_positions = {pos for path in best_paths for pos in path}
grid_with_shortest_paths = {k: "O" if k in best_path_positions else v for k, v in grid.items()}
print(f"There are {len(best_paths)} paths with the best score of {minimum_path_score}.")
print_grid(grid_with_shortest_paths)

In [ ]:
# Submit answer
submit(len(best_path_positions), part="b", day=16, year=2024)